In [1]:
#!conda install pandas numpy statsmodels openpyxl

In [2]:
from itertools import product
from typing import List, Tuple

import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.formula.api import ols


def gen_plan(*factors: Tuple[str, List], res_str: str, replications=3, save_path: str):
    assert(len(factors) > 0)
    assert(replications > 1)
    
    factor_names, factor_levels = zip(*factors)
    cp = np.array(list(product(*factor_levels, range(replications))))
    df_dict = {}
    df_dict[res_str] = ""
    for i in range(len(factor_names)):
        df_dict[factor_names[i]] = cp[:, i]
    df = pd.DataFrame(df_dict)
    df.to_excel(save_path, index=False)

def run_plan(save_path: str, sheet_name=0) -> sm.stats.anova_lm:
    df = pd.read_excel(save_path, sheet_name=sheet_name)
    key_it = iter(df.keys())
    modelString = f"{next(key_it)} ~ C({next(key_it)})"
    for key in key_it:
        modelString += f"*C({key})"
    model = ols(modelString, df).fit()

    return sm.stats.anova_lm(model)

# example usage:
# gen_plan(('Processors', ['A', 'B']), ('Workloads', ['C', 'D']), res_str="Code_Size", replications=2, save_path='test.xlsx')
# fill in experiments results in spreadsheet
# run_plan('test.xlsx', 0)

# 2^k Factorial
### "executorMemory" fixed at 8Gi for all experiments

## FASHIONMNIST, FASHIONMNISTCNN

In [3]:
run_plan('fashion-2k.xlsx', 'AvgProcessorUtil')

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,19.859178,19.859178,15.828473,1.079753e-03
C(executorCores),1.0,3693.421918,3693.421918,2943.789068,1.435550e-19
C(batchSize),1.0,27.046174,27.046174,21.556766,2.707718e-04
C(maxEpoch),1.0,0.004680,0.004680,0.003730,9.520546e-01
C(dataParallelism):C(executorCores),1.0,1.365791,1.365791,1.088584,3.122938e-01
C(dataParallelism):C(batchSize),1.0,2.498171,2.498171,1.991132,1.773711e-01
C(executorCores):C(batchSize),1.0,18.955864,18.955864,15.108500,1.309238e-03
C(dataParallelism):C(maxEpoch),1.0,0.010549,0.010549,0.008408,9.280795e-01
C(executorCores):C(maxEpoch),1.0,0.004680,0.004680,0.003730,9.520546e-01
C(batchSize):C(maxEpoch),1.0,0.792855,0.792855,0.631934,4.382806e-01


In [4]:
run_plan('fashion-2k.xlsx', 'AvgAccuracy')

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,1.241415e+02,1.241415e+02,1.828161e+27,6.759949e-210
C(executorCores),1.0,8.077936e-28,8.077936e-28,1.189591e-02,9.145044e-01
C(batchSize),1.0,1.386113e+00,1.386113e+00,2.041248e+25,2.798286e-194
C(maxEpoch),1.0,4.437221e+00,4.437221e+00,6.534439e+25,2.537417e-198
C(dataParallelism):C(executorCores),1.0,1.817536e-27,1.817536e-27,2.676580e-02,8.720927e-01
C(dataParallelism):C(batchSize),1.0,1.522513e+00,1.522513e+00,2.242117e+25,1.320666e-194
C(executorCores):C(batchSize),1.0,5.048710e-29,5.048710e-29,7.434944e-04,9.785839e-01
C(dataParallelism):C(maxEpoch),1.0,4.970045e-01,4.970045e-01,7.319099e+24,1.024230e-190
C(executorCores):C(maxEpoch),1.0,1.029476e-27,1.029476e-27,1.516050e-02,9.035392e-01
C(batchSize):C(maxEpoch),1.0,1.428050e-02,1.428050e-02,2.103007e+23,2.204615e-178


In [5]:
run_plan('fashion-2k.xlsx', 'AvgTrainTime')

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,4.677409e+10,4.677409e+10,14815.444678,3.604119e-25
C(executorCores),1.0,1.802433e+10,1.802433e+10,5709.110115,7.316912e-22
C(batchSize),1.0,5.867128e+08,5.867128e+08,185.838161,3.176220e-10
C(maxEpoch),1.0,7.906012e+09,7.906012e+09,2504.187248,5.197865e-19
C(dataParallelism):C(executorCores),1.0,4.735585e+09,4.735585e+09,1499.971336,3.037928e-17
C(dataParallelism):C(batchSize),1.0,2.967698e+06,2.967698e+06,0.940003,3.467062e-01
C(executorCores):C(batchSize),1.0,1.322009e+08,1.322009e+08,41.873919,7.720474e-06
C(dataParallelism):C(maxEpoch),1.0,2.263073e+09,2.263073e+09,716.816260,1.024128e-14
C(executorCores):C(maxEpoch),1.0,7.801513e+08,7.801513e+08,247.108798,3.776148e-11
C(batchSize):C(maxEpoch),1.0,1.007466e+07,1.007466e+07,3.191096,9.300180e-02


## CIFAR10, CIFAR10CNN

In [6]:
run_plan('cifar10-2k.xlsx', 'AvgProcessorUtil')

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,16.940565,16.940565,21.680473,2.634734e-04
C(executorCores),1.0,4167.000518,4167.000518,5332.911877,1.260414e-21
C(batchSize),1.0,17.244596,17.244596,22.069571,2.419248e-04
C(maxEpoch),1.0,0.250809,0.250809,0.320984,5.788781e-01
C(dataParallelism):C(executorCores),1.0,2.702231,2.702231,3.458306,8.142194e-02
C(dataParallelism):C(batchSize),1.0,5.879878,5.879878,7.525046,1.443494e-02
C(executorCores):C(batchSize),1.0,4.359366,4.359366,5.579101,3.118520e-02
C(dataParallelism):C(maxEpoch),1.0,0.183467,0.183467,0.234800,6.345541e-01
C(executorCores):C(maxEpoch),1.0,0.001785,0.001785,0.002284,9.624702e-01
C(batchSize):C(maxEpoch),1.0,0.010476,0.010476,0.013408,9.092592e-01


In [7]:
run_plan('cifar10-2k.xlsx', 'AvgAccuracy')

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,1.279371e+03,1.279371e+03,4.893178e+28,2.566437e-221
C(executorCores),1.0,3.155444e-28,3.155444e-28,1.206855e-02,9.138889e-01
C(batchSize),1.0,8.972641e+01,8.972641e+01,3.431744e+27,4.384692e-212
C(maxEpoch),1.0,1.177499e+02,1.177499e+02,4.503551e+27,4.984461e-213
C(dataParallelism):C(executorCores),1.0,2.473868e-27,2.473868e-27,9.461743e-02,7.623550e-01
C(dataParallelism):C(batchSize),1.0,1.294762e+02,1.294762e+02,4.952046e+27,2.332281e-213
C(executorCores):C(batchSize),1.0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
C(dataParallelism):C(maxEpoch),1.0,2.464020e-01,2.464020e-01,9.424078e+24,1.355650e-191
C(executorCores):C(maxEpoch),1.0,5.129568e-28,5.129568e-28,1.961894e-02,8.903547e-01
C(batchSize):C(maxEpoch),1.0,2.450000e-03,2.450000e-03,9.370456e+22,1.418968e-175


In [8]:
run_plan('cifar10-2k.xlsx', 'AvgTrainTime')

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,2.298950e+12,2.298950e+12,23105.583160,1.032884e-26
C(executorCores),1.0,8.249171e+11,8.249171e+11,8290.825188,3.723488e-23
C(batchSize),1.0,3.729589e+06,3.729589e+06,0.037484,8.489190e-01
C(maxEpoch),1.0,2.881030e+11,2.881030e+11,2895.577187,1.637176e-19
C(dataParallelism):C(executorCores),1.0,2.780529e+11,2.780529e+11,2794.569394,2.171729e-19
C(dataParallelism):C(batchSize),1.0,1.092747e+08,1.092747e+08,1.098265,3.102208e-01
C(executorCores):C(batchSize),1.0,8.760180e+07,8.760180e+07,0.880441,3.620305e-01
C(dataParallelism):C(maxEpoch),1.0,9.840455e+10,9.840455e+10,989.014390,8.161604e-16
C(executorCores):C(maxEpoch),1.0,3.384857e+10,3.384857e+10,340.194891,3.322069e-12
C(batchSize):C(maxEpoch),1.0,2.817115e+07,2.817115e+07,0.283134,6.019657e-01


# 2-Factor Full Factorial
### "executorMemory" fixed at 8Gi for all experiments
### "dataParallelism" fixed at 4 for all experiments
### "maxEpoch" fixed at 4 for all experiments

## FASHIONMNIST, FASHIONMNISTCNN

In [9]:
run_plan('fashion-full.xlsx', 'AvgProcessorUtil')

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,3620.011449,1810.005724,311.735779,6.728009e-18
C(batchSize),3.0,28.444028,9.481343,1.632964,2.080621e-01
C(executorCores):C(batchSize),6.0,22.067002,3.677834,0.633430,7.022450e-01
Residual,24.0,139.349219,5.806217,NaN,NaN


In [10]:
run_plan('fashion-full.xlsx', 'AvgAccuracy')

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,3.685558e-27,1.842779e-27,6.186441e-02,9.401597e-01
C(batchSize),3.0,2.616355e+01,8.721183e+00,2.927811e+26,6.979365e-307
C(executorCores):C(batchSize),6.0,4.723699e-27,7.872832e-28,2.643008e-02,9.999025e-01
Residual,24.0,7.148973e-25,2.978739e-26,NaN,NaN


In [11]:
run_plan('fashion-full.xlsx', 'AvgTrainTime')

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,5.093723e+09,2.546861e+09,16310.952061,2.492244e-38
C(batchSize),3.0,1.000924e+09,3.336413e+08,2136.750261,2.917843e-29
C(executorCores):C(batchSize),6.0,1.760206e+08,2.933677e+07,187.882490,5.894695e-19
Residual,24.0,3.747462e+06,1.561442e+05,NaN,NaN


## CIFAR10, CIFAR10CNN

In [12]:
run_plan('cifar10-full.xlsx', 'AvgProcessorUtil')

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,3702.217611,1851.108806,603.133865,3.037661e-21
C(batchSize),3.0,25.091479,8.363826,2.725127,6.649304e-02
C(executorCores):C(batchSize),6.0,28.850295,4.808383,1.566682,2.000053e-01
Residual,24.0,73.659620,3.069151,NaN,NaN


In [13]:
run_plan('cifar10-full.xlsx', 'AvgAccuracy')

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,1.009742e-28,5.048710e-29,5.852231e-03,0.994166
C(batchSize),3.0,2.753125e+02,9.177083e+01,1.063765e+28,0.000000
C(executorCores):C(batchSize),6.0,8.361926e-28,1.393654e-28,1.615460e-02,0.999977
Residual,24.0,2.070476e-25,8.626983e-27,NaN,NaN


In [14]:
run_plan('cifar10-full.xlsx', 'AvgTrainTime')

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,1.304330e+11,6.521648e+10,7158.868486,4.822962e-34
C(batchSize),3.0,2.781781e+09,9.272604e+08,101.786167,8.722440e-14
C(executorCores):C(batchSize),6.0,8.233005e+08,1.372167e+08,15.062400,4.312979e-07
Residual,24.0,2.186373e+08,9.109886e+06,NaN,NaN
